In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# ignore warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.__version__

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv")

In [ ]:
for df in [train, test]:
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True, drop=False)

In [ ]:
train.head()

In [ ]:
train.isna().sum()

In [ ]:
country = train['country'].unique()
print('Unique value of country column: ',country)

# EDA and Feature Engineering

In [ ]:
train.value_counts(train['country']).plot.bar()
plt.title('Data Distribution')
plt.xlabel('Country')
plt.ylabel('counts')

plt.show()

In [ ]:
store = train['store'].unique()
print('Unique value of store column: ', store)


train.value_counts(train['store']).plot.bar()
plt.title('Data Distribution')
plt.xlabel('Store')
plt.ylabel('counts')

plt.show()

In [ ]:
product = train['product'].unique()
print('Unique value of product column: ', product)

train.value_counts(train['product']).plot.bar()
plt.title('Data Distribution')
plt.xlabel('Product')
plt.ylabel('counts')

plt.show()

In [ ]:
train['date'] = pd.to_datetime(train['date'])

train.info()

In [ ]:
# for every store, country and product combination (18 in total), we have data for 1461 days
train.groupby(['country', 'store', 'product']).count()

In [ ]:
print("Training:")
print(f"First day: {train.date.min()}")
print(f"Last day: {train.date.max()}")

In [ ]:
print("Predictions:")
print(f"First day: {test.date.min()}")
print(f"Last day: {test.date.max()}")

In [ ]:
# num_sold is never zero, every product is always sold every day in every store
train.groupby(['country', 'store', 'product']).num_sold.agg(['min', 'max', 'mean'])

In [ ]:
# On average, KaggleRama sells more of every product in every country in every year
train.groupby(['country', 'store', 'product', train.date.dt.year]).num_sold.mean().unstack('store')

In [ ]:
# group products by year
grp = train.groupby(['country', 'store', 'product', train.date.dt.year]).num_sold.mean().unstack(level='product')
grp['Hat/Mug'] = grp['Kaggle Hat'] / grp['Kaggle Mug']
grp['Mug/Sticker'] = grp['Kaggle Mug'] / grp['Kaggle Sticker']

In [ ]:
# ratios remain about constant, the ratio of Hat:Mug ~ 1.77 and Mug/Sticker ~ 1.98
grp

In [ ]:
# group products by month
grp = train.groupby(['product', train.date.dt.month]).num_sold.mean().unstack(level='product')
grp['Hat/Mug'] = grp['Kaggle Hat'] / grp['Kaggle Mug']
grp['Mug/Sticker'] = grp['Kaggle Mug'] / grp['Kaggle Sticker']

In [ ]:
# the ratio doesn't remain the same for months, so seasonality is there for products
grp

In [ ]:
# Sale is higher on weekdays, (here, 0=Monday and 6=Sunday)
train.groupby(['country', 'store', 'product', train.date.dt.weekday]).num_sold.mean().unstack('product')

### Histograms

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax.hist(df.num_sold, bins=50, color='blue')
    ax.set_title(combi)
    
plt.suptitle('Histograms of number of products sold', y=1.0)
plt.tight_layout(h_pad=0.3)
plt.show()

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train.groupby(['country', 'store','product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax.plot(df.num_sold)
    ax.set_title(combi)
    
plt.tight_layout(h_pad=3.0)
plt.suptitle('Daily sales for 2015-18', y=1.03)
plt.show()

Inference: 
* 1st column: Kaggle Hats sell more in the first half of every year 
* 2nd column: Kaggle mugs sell more at the end of every year (during winters)
* 3rd column: Kaggle stickers sale remains constant throughout the year

We zoom in on the end of year, particularly December and plot sales average for these 31 days:

In [ ]:
# for December
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax.bar(range(1, 32), df.num_sold[df.date.dt.month==12].groupby(df.date.dt.day).mean(), color = ['r']*25 + ['orange']*6)
    ax.set_title(combi)
    ax.set_xticks(ticks=range(5, 31, 5))
plt.tight_layout(h_pad=3.0)
plt.suptitle('Daily sales of December', y=1.03)
plt.show()

### Trend and Seasonality

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    resampled = df.resample('MS').num_sold.sum()
    ax.bar(range(len(resampled)), resampled)
    ax.set_title(combi)
    ax.set_ylim(resampled.min(), resampled.max())
    ax.set_xticks(range(0, 48, 12), [f"Jan{y}" for y in range(2015, 2019)])
plt.suptitle('Monthly sales for 2015-2018', y=1.03)
plt.tight_layout(h_pad=3.0)
plt.show()

Inference:

* We can see an upward trend
* There is seasonal variation for every product

In [ ]:
# seeing the seasonality more clearly by averaging over month
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    resampled = df.resample('MS').sum()
    resampled = resampled.groupby(resampled.index.month).mean()
    ax.bar(range(1, 13), resampled.num_sold)
    ax.set_xticks(ticks=range(1, 13), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.set_title(combi)
    ax.set_ylim(resampled.num_sold.min(), resampled.num_sold.max())
plt.suptitle('Monthly sales for 2015-2018', y=1.03)
plt.tight_layout(h_pad=3.0)
plt.show()

Inference:

* Kaggle Hat: max in Apr, min in Sep
* Kaggle Mug: max in Dec, min in Jun,Jul
* Kaggle Sticker: max in Dec, min in Feb

In [ ]:
# weekly variation
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax.bar(range(1, 8), df.num_sold.groupby(df.date.dt.weekday).mean(), color=['r']*5+ ['orange']*2)
    ax.set_title(combi)
    ax.set_xticks(ticks=range(1, 8), labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.suptitle('Weeky average')
plt.tight_layout(h_pad=3.0)
plt.show()

Inference:

* Saturday and Sundays experience higher sales
* Fri also has larger sales compared to other working days

### Easter holiday

The diagram shows daily sales for April of the four years and we find that during the week after Easter, sales are higher than normal. Weekends are colored orange, Easter Sunday is marked red, the week after Easter is colored light blue.

In [ ]:
import dateutil.easter as easter

In [ ]:
plt.figure(figsize=(12, 8))
for i, (year, df) in enumerate(train.groupby(train.date.dt.year)):
    df = df.reset_index(drop=True)
    ax = plt.subplot(4, 1, i+1, ymargin=0.5)
    april = df.num_sold[(df.date.dt.month==4)].groupby(df.date.dt.day).mean()
    date_range = pd.date_range(start=f'{year}-04-01', end=f'{year}-04-30', freq='D')
    easter_date = easter.easter(year)
    color = ['r' if d == easter_date else 'lightblue' if (d.date() - easter_date).days in range(6) else 'b' if d.dayofweek < 5 else 'orange' for d in date_range]
    ax.bar(range(1, 31),
           april,
           color=color)
    ax.set_title(str(year))
    ax.set_xticks(ticks=range(5, 31, 5))
plt.tight_layout(h_pad=3.0)
plt.suptitle('Daily sales for April', y=1.03)
plt.show()

In [ ]:
df_train = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv")
df_test = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv")
df_train

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

In [ ]:
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day

df_test['year'] = df_test['date'].dt.year
df_test['month'] = df_test['date'].dt.month
df_test['day'] = df_test['date'].dt.day

df_train

In [ ]:
sns.lineplot(data=df_train, x="year", y="num_sold", palette = "coolwarm", hue = 'store');

In [ ]:
#rename month column for prettier x axis
df_train_copy = df_train.copy()
df_train_copy['month'].replace({1: "Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun", 7: "Jul", 8:"Aug", 9:"Sep", 10: "Oct", 11: "Nov", 12:"Dec"}, inplace = True)

#Create subplots
fig, ax = plt.subplots(1,2, sharey=True, figsize=(25,5))

# adjust the suptitle for the subplots
plt.subplots_adjust(top=0.82, left = 0.42)

#remove grid
sns.despine()

fig.suptitle("KaggleMart VS KaggleRama Sales/Year", size = 15)

#add plot labels and position them to the left
ax[0].set_title("KaggleMart", loc = 'left')
ax[1].set_title("KaggleRama", loc = 'left')

#create lineplots
sns.lineplot(x='month', y='num_sold', hue='year', ci = None, legend = False, marker = 'o', palette = "coolwarm", data=df_train_copy.loc[df_train_copy['store'] == 'KaggleMart'], ax = ax[0]);
sns.lineplot(x='month', y='num_sold', hue='year', ci = None, marker = 'o', palette = "coolwarm", data=df_train_copy.loc[df_train_copy['store'] == 'KaggleRama'], ax = ax[1]);

#to reduce legend duplication
ax[1].legend(loc=(1.1, 0.5));

In [ ]:
# drop date column
df_train.drop('date', axis = 1, inplace = True)
df_test.drop('date', axis = 1, inplace = True)

In [ ]:
df_train.head()

## ***Modelling***

In [ ]:
# One hot encoder
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [ ]:
# Add back the row_id
df_train['row_id'] = df_train.index
df_test['row_id'] = df_test.index

X_train = df_train.drop('num_sold', axis = 1)
y_train = df_train['num_sold']

In [ ]:
from sklearn.model_selection import train_test_split

tf.random.set_seed(42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

# 1. Linear Regression

In [ ]:

from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

y_pred_lr = reg.predict(df_test)

In [ ]:
reg.score(df_test,y_pred_lr)

In [ ]:
# dataframe=pd.DataFrame({"row_id":df['row_id'],'num_sold':y_pred})
output = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
output['num_sold']= y_pred_lr

#idea comes from https://www.kaggle.com/c/tabular-playground-series-jan-2022/discussion/299162
output['num_sold'] = output['num_sold'].apply(np.ceil)
output.to_csv('linreg_submission.csv',index =False)

In [ ]:
output

# 2. XGBoost Regressor

In [ ]:
from xgboost import XGBRegressor 
# define model
model = XGBRegressor(colsample_bytree = 0.7,
                     learning_rate = 0.07,
                     max_depth= 10,
                     min_child_weight= 4,
                     n_estimators = 700,
                     subsample = 0.7)
# fit model
model.fit(X_train, y_train)
# make a prediction
y_pred_xgb = model.predict(df_test)

In [ ]:
# dataframe=pd.DataFrame({"row_id":df['row_id'],'num_sold':y_pred})
output = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
output['num_sold']= y_pred_xgb

#idea comes from https://www.kaggle.com/c/tabular-playground-series-jan-2022/discussion/299162
output['num_sold'] = output['num_sold'].apply(np.ceil)
output.to_csv('xgboost_submission.csv',index =False)

In [ ]:
output

# 3. CNN

In [ ]:
tf.random.set_seed(42)

# 1. Create a model
model_1 = tf.keras.Sequential([
           tf.keras.layers.Dense(500, activation='relu'),
           tf.keras.layers.Dense(400, activation='relu'),
           tf.keras.layers.Dense(300, activation='relu'),
           tf.keras.layers.Dense(250, activation='relu'),
           tf.keras.layers.Dense(100, activation='relu'),
           tf.keras.layers.Dense(50, activation='relu'),
           tf.keras.layers.Dense(10, activation='relu'), 
           tf.keras.layers.Dense(1)
])

# 2. Comile the model
model_1.compile(loss=tf.keras.losses.mae,
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                 metrics=['MAE'])

# 3. Fit the model
history = model_1.fit(X_train, 
                      y_train, 
                      epochs=50,
                      verbose = 1,
                      validation_data=(X_test, y_test))

In [ ]:
model_1.summary()

In [ ]:
y_pred_cnn = model_1.predict(df_test)


In [ ]:
# dataframe=pd.DataFrame({"row_id":df['row_id'],'num_sold':y_pred})
output = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
output['num_sold']= y_pred_cnn

#idea comes from https://www.kaggle.com/c/tabular-playground-series-jan-2022/discussion/299162
output['num_sold'] = output['num_sold'].apply(np.ceil)
output.to_csv('cnn_submission.csv',index =False)

In [ ]:
output

# 4. CNN more layers

In [ ]:
tf.random.set_seed(42)

# 1. Create a model
model_2 =tf.keras.Sequential([
           tf.keras.layers.Dense(1000, activation='relu'),
           tf.keras.layers.Dense(500, activation='relu'),
           tf.keras.layers.Dense(400, activation='relu'),
           tf.keras.layers.Dense(300, activation='relu'),
           tf.keras.layers.Dropout(0.2),
           tf.keras.layers.Dense(250, activation='relu'),
           tf.keras.layers.Dropout(0.2),
           tf.keras.layers.Dense(100, activation='relu'),
           tf.keras.layers.Dense(50, activation='relu'),
           tf.keras.layers.Dense(10, activation='relu'), 
           tf.keras.layers.Dense(1)
])

# 2. Comile the model
model_2.compile(loss=tf.keras.losses.mae,
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                 metrics=['MAE'])



In [ ]:
# 3. Fit the model
history = model_2.fit(X_train, 
                      y_train, 
                      epochs=50,
                      verbose = 1,
                      validation_data=(X_test, y_test))

In [ ]:
model_2.summary()

In [ ]:
y_pred_cnn2 = model_2.predict(df_test)



In [ ]:
# dataframe=pd.DataFrame({"row_id":df['row_id'],'num_sold':y_pred})
output = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
output['num_sold']= y_pred_cnn2

#idea comes from https://www.kaggle.com/c/tabular-playground-series-jan-2022/discussion/299162
output['num_sold'] = output['num_sold'].apply(np.ceil)
output.to_csv('cnn_submission2.csv',index =False)